In [ ]:
import torch
import requests
from PIL import Image
from transformers import AutoModelForCausalLM, LlamaTokenizer
import boto3
from torchvision import transforms
#import deepspeed
import json
from yarn_test import Yarn
from io import BytesIO

from typing import Optional, List

import json

# Load the configuration file
config_path = 'config.json'
with open(config_path, 'r') as config_file:
    config_dict = json.load(config_file)


from modeling_cogvlm import CogVLMForCausalLM, CogVLMConfig

In [ ]:
# import boto3
# import os

# # Define the S3 bucket and object key
# bucket_name = 'mock-cogvlm-test'
# object_key = 'model/mino_chat_exp5.bin'

# # Create a S3 client
# s3_client = boto3.client('s3')

# # Specify the path to save the model on your SageMaker instance
# local_model_path = '/home/ec2-user/SageMaker/testing/mino_chat_exp5.bin'

# # Download the model from S3 to your notebook environment
# s3_client.download_file(bucket_name, object_key, local_model_path)

# # If the model is a PyTorch model, you might load it like this:
# # (Make sure you have the correct environment and dependencies installed)

# # import torch
# # model = torch.load(local_model_path, map_location=torch.device('cpu'))

# # If it's a TensorFlow model, it might look like this:
# # from tensorflow.keras.models import load_model
# # model = load_model(local_model_path)

# # Now you can use the model for inference


In [ ]:
LANGUAGE_TOKEN_TYPE = 0
VISION_TOKEN_TYPE = 1


def calculate_num_image_tokens(image_size, patch_size):
    num_patches_per_side = image_size // patch_size
    return (num_patches_per_side ** 2) + 2

In [ ]:
# def inference(model, tokenizer, transform, query, image_bytes):
    
    
    
#     image = Image.open(BytesIO(image_bytes)).convert('RGB')
    
    #image = [transform(image)]
    
    #images_processed = []
    
#     for img_list in image:
#         processed_imgs = [img.to('cuda').half() for img in img_list]
#         images_processed.append(processed_imgs)
    
    #image = image.to('cuda').half()
    
#     input_ids = [tokenizer.bos_token_id]
#     token_type_ids = [LANGUAGE_TOKEN_TYPE]

#     vision_token_num = calculate_num_image_tokens(image_size=224, patch_size=14)

#     # Add vision tokens placeholder for each image
#     input_ids += [tokenizer.pad_token_id] * vision_token_num
#     token_type_ids += [VISION_TOKEN_TYPE] * (vision_token_num)

#     # Tokenize the question and append
#     query_ids = tokenizer(query, truncation=True, max_length=2048, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze(0)
#     input_ids += query_ids.tolist()
#     token_type_ids += [LANGUAGE_TOKEN_TYPE] * len(query_ids)
    
#     attention_mask = [1] * len(input_ids)
    
#     attention_mask  = torch.tensor(attention_mask, dtype=torch.long).unsqueeze(0).to('cuda')
    
#     input_ids = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).to('cuda')
#     token_type_ids = torch.tensor(token_type_ids, dtype=torch.long).unsqueeze(0).to('cuda')
    
    
#     with torch.no_grad():
#         outputs = model.generate(input_ids=input_ids, token_type_ids=token_type_ids, images=images_processed, attention_mask=attention_mask)
        
       
#         generated_ids = outputs[0]  

#         # Decode the token ids to text
#         generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
#         print("Generated text:", generated_text)

    
    
    
    
    

    
    
    
    
    
    
    

In [ ]:
def adjust_position_embedding_size(model, new_num_positions=257):
    vision_model = model.vision  # Adjust this path based on your model's structure
    for layer in vision_model.children():
        if isinstance(layer, PatchEmbedding):  
            new_position_embedding = torch.nn.Embedding(new_num_positions, layer.position_embedding.embedding_dim)
            # Copy existing weights for positions that overlap
            if new_num_positions <= layer.position_embedding.weight.size(0):
                new_position_embedding.weight.data[:new_num_positions] = layer.position_embedding.weight.data[:new_num_positions]
            else:
                raise ValueError("New number of positions exceeds the size of the existing weights")
            layer.position_embedding = new_position_embedding
            break  # Assuming there's only one PatchEmbedding layer to modify


In [ ]:
def replace_rotary_embeddings(model):
    # Iterate over all named modules in the model
    for name, module in model.named_modules():
        # Check if the current module has a 'rotary_emb' attribute
        if hasattr(module, 'rotary_emb'):
            # Calculate the new dimension based on model configuration
            dim = model.config.hidden_size // model.config.num_attention_heads
            
            # Instantiate your replacement embedding
            yarn_rope = Yarn(dim)
            
            # Replace the rotary_emb with yarn_rope
            setattr(module, 'rotary_emb', yarn_rope)


In [ ]:

torch.cuda.empty_cache()

tokenizer = LlamaTokenizer.from_pretrained('lmsys/vicuna-7b-v1.5') 


config_path = 'config.json'
with open(config_path, 'r') as config_file:
    config_dict = json.load(config_file)

# Create the configuration object for your model
config = CogVLMConfig(**config_dict)

# Initialize your model with the configuration
model = CogVLMForCausalLM(config)
#print(model)


# replace_rotary_embeddings(model)
# adjust_position_embedding_size(model, new_num_positions=257)
# Load the model weights
model_state_dict = torch.load("mino_chat_exp9.bin")

#state_dict = {k.replace("model.", ""): v for k, v in model_state_dict.items()}
#print(f'state_dict:{model_state_dict.keys()}')
model.load_state_dict(model_state_dict)
#print(f'model after:{model}')

# new_num_positions = 257
# config.vision_config['num_positions'] = new_num_positions

# #Directly modify the position_embedding in the PatchEmbedding layer
# vision_model = model.vision
# for layer in vision_model.children():
#     if layer.__class__.__name__ == 'PatchEmbedding':
#         new_position_embedding = torch.nn.Embedding(new_num_positions, layer.position_embedding.embedding_dim)
#         new_position_embedding.weight.data[:257] = layer.position_embedding.weight.data[:257]
#         layer.position_embedding = new_position_embedding


# Move model to GPU and set it to evaluation mode
#adjust_position_embedding_size(model, new_num_positions=257)
model.to('cuda').bfloat16().eval()



In [ ]:
# inference(model=model, tokenizer=tokenizer, transform=transform, query=query, image_bytes=image_bytes)

In [ ]:
with open('41_jpg.rf.7c258d236d88a4304878a19405b61b5c.jpg', 'rb') as f:
    image_bytes = f.read()

In [ ]:
image = Image.open(BytesIO(image_bytes)).convert('RGB')
# assistant_text1 = """This image depicts a Low squamous intra-epithelial lesion, an indicator of initial changes in cervical cells that are usually not a cause for immediate concern. """
# user_text1 = """Give further reasons"""

# assistant_text2 = """Low squamous intra-epithelial lesion is usually not a cause for immediate concern, but it can still be a precursor to cancer.""" 

# user_text2 = """what are the causes of low squamous intra-epithelial?"""
 
inputs = model.build_conversation_input_ids(tokenizer, history=[], query="USER: what medical image is this? ASSISTANT:", 
                                            images=[image], 
                                            template_version='chat')  

# inputs = model.build_conversation_input_ids(tokenizer, history=[], query=f"USER: Is this low squamous or high squamous? ASSISTANT: {assistant_text1} USER: {user_text1} ASSISTANT: {assistant_text2} USER: {user_text2} ASSISTANT:",  
#                                             images=[image], 
#                                             template_version='chat')  

inputs = {
    'input_ids': inputs['input_ids'].unsqueeze(0).to('cuda'),
    'token_type_ids': inputs['token_type_ids'].unsqueeze(0).to('cuda'),
    'attention_mask': inputs['attention_mask'].unsqueeze(0).to('cuda'),
    'images': [[inputs['images'][0].to('cuda').to(torch.bfloat16)]],
}
gen_kwargs = {"max_length": 2048, "do_sample": False}

with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    outputs = tokenizer.decode(outputs[0])
    outputs = outputs.split("</s>")[0]
    print(outputs)

In [ ]:
print(torch.load('mino_chat_exp6.bin'))

In [ ]:
!deepspeed inference.py --deepspeed --deepspeed_config ds_inference_config.json


In [ ]:
!pip install boto3


In [ ]:
torch.cuda.empty_cache()

In [ ]:
#41_jpg.rf.7c258d236d88a4304878a19405b61b5c.jpg

#Glioma_T1_glioma (1).jpeg

#dataset_frame064_stack_fov008.png

In [ ]:
#What are the chances of bladder cancer recurrence within 1 months for a patient who had 1 tumors of placebo cm size, treated with 7 after undergoing transurethral surgical excision?